In [ ]:
/************************************************************************
 * Last Run Date: 24 April 2017	            							*
 * Program Name: panders2_hw12_prog.sas									*
 * Program Location: C:/Users/Philip/Schools/TAMU/STAT_657/				*
 *				SASUniversityEdition/myfolders/homework/git_hw/hw_13/sasprogs/	* 
 * Creation Date: 24 April 2017											*
 * Author: Philip Anderson												*
 * Purpose: Meeting objectives for Assignment 13						*
 * Inputs: basketball data         *
 * Output: panders2_hw13_output.pdf															*
 * Modification History: *
 * Execution Instructions: N/A - run as-is												*
*/**********************************************************************;

In [1]:
/*specify the option that will print information about the indexes */
options msglevel=i nonumber nodate;
libname bball '/folders/myfolders/homework/git_hw/hw_04/sasdata' access=readonly;
libname sasdata '/folders/myfolders/homework/git_hw/hw_13/sasdata';


In [22]:
/*Question 1 - recreate the output of p306d02 using SQL */
/*SAS is as follows: */
/* data country;
   keep Start Label FmtName;
   retain FmtName '$Country';
   set orion.Country(rename=(Country=Start
                              Country_Name = Label));
run;
 */
 
proc sql;
create table test as
select 
distinct
'$seedz' as FmtName 
, team as Start
, put(seed_, 2.) as Label
from
bball.ncaam04
;

insert into test
    values ('$seedz', 'other', 'NA')
;
quit;

proc print data=test; run; 


Obs,FmtName,Start,Label
1,$seedz,Boston College,6
2,$seedz,Connecticut,2
3,$seedz,Duke,1
4,$seedz,Georgia Tech,3
5,$seedz,Gonzaga,2
6,$seedz,Kansas,4
7,$seedz,Kentucky,1
8,$seedz,Maryland,4
9,$seedz,Mississippi State,2
10,$seedz,North Carolina,6


In [27]:
proc format library=sasdata.myfmts cntlin=test fmtlib;
    select $seedz;
run;

options fmtsearch=(sasdata sasdata.myfmts);


In [4]:
/* Question 2 - create a picture format that can be applied to the PPG column 
allow space for PPG to be displayed as two digits along with a single decimal place (00.0)*/

/* proc print data=bball.ncaam04 (obs=10); run; */

proc format;
    picture ppg_frmtr
    15 - high = '(00.0)' (prefix='High(')
    7.7 -< 15 = '(00.0)' (prefix='Medium(')
    low -< 7.7 = '(00.0)' (prefix='Low(')
    ;
run;



In [2]:
/*Question 3 - use a SAS procedure to place a copy of the ncaam06 dataset in the WORK library */
proc datasets noprint;
copy in=bball out=work;
select ncaam06;
quit;



In [35]:
proc sql;
create table ncaam06_update as
select 
ncaam06.*
, put(ppg, 12.) as ppgrate
, '  ' as seed04 
from
ncaam06
;

alter table ncaam06_update
    modify School char(19);

create table ncaam06_update_b as
select 
ncaam06_update.*
, put(ppg, ppg_frmtr.) as ppg_b 
, put(seed04, $seedz.) as seed04
, (case
    when School = 'Indania' then 'Indiana'
    when School = 'Boston Coll' then 'Boston College'
    when School = 'George mason' then 'George Mason'
    when School = 'Oral Robt-16' then 'Oral Roberts'
    when School = 'Wisc. Milwaukee' then 'Wisconsin Milwaukee'
    else School end) as School
from
ncaam06_update
;
quit;

proc print data=ncaam06_update_b (obs=10); run;
proc contents data=ncaam06_update_b; run;

Obs,RPI,Seed,School,Region,Teamdex,Player,GP,PPG,RPG,APG,TotPts,K_factor,P_Rank,ppgrate,seed04,ppg_b
1,0.5692,13,Air Force,WDC,64.92,Jacob Burtschi,30,12.4,6.1,2.6,372.0,95.920,183,12,,Medium(12.4)
2,0.5636,10,Alabama,OAK,70.36,Jermareo Davidson,29,14.1,9.0,1.1,408.9,104.905,146,14,,Medium(14.1)
3,0.5274,16,Albany,WDC,54.74,Jamar Wilson,31,17.7,5.0,3.6,548.7,99.875,170,18,,High(17.7)
4,0.5869,8,Arizona,MIN,76.69,Hassan Adams,29,17.3,5.1,2.9,501.7,119.075,64,17,,High(17.3)
5,0.5667,8,Arkansas,OAK,74.67,Ronnie Brewer,31,18.5,4.8,3.2,573.5,121.845,53,19,,High(18.5)
6,0.5364,15,Belmont,OAK,57.64,Justin Hare,30,15.8,4.0,2.9,474.0,97.140,177,16,,High(15.8)
7,0.5897,4,Boston Coll,MIN,84.97,Craig Smith,33,17.3,9.0,2.9,570.9,130.815,22,17,,High(17.3)
8,0.5897,4,Boston Coll,MIN,84.97,Jared Dudley,33,16.5,6.8,3.2,544.5,128.695,28,17,,High(16.5)
9,0.5897,4,Boston Coll,MIN,84.97,Sean Marshall,33,11.5,4.0,2.3,379.5,115.445,82,12,,Medium(11.5)
10,0.5897,4,Boston Coll,MIN,84.97,Tyrese Rice,33,9.2,1.4,2.5,303.6,109.350,114,9,,Medium(9.2)


In [40]:

/*Question 5 - create a composite index for ncaam06 using player, school, and region-  in that order */
data ncaam06_update_b 
    (index=(bball_idx=(player school region )));
set ncaam06_update;
run;

proc contents data=ncaam06_update_b; run;

In [47]:
/*Question 6 - create a print procedure that will print the columns shown in the output */

/* index was not used - this is likely the most efficient action - calling the index could tie up more resources than not using it */
title '6a. IDXWHERE on Player';
proc print data=ncaam06_update_b (idxwhere=yes keep=player school region seed ppg seed);
where player in ('Steve Burtt', 'Jared Dudley', 'Stanley Burrell');
run;

/**/
title '6b. IDXWHERE on School';
proc print data=ncaam06_update_b (idxwhere=yes keep=player school region seed ppg seed);
where school='Texas';
run;

/**/
title '6c. IDXNAME on School';
proc print data=ncaam06_update_b (idxname=bball_idx keep=player school region seed ppg seed);
where school='Texas';
run;

/**/
title '6c. IDXNAME on School';
proc print data=ncaam06_update_b (idxname=bball_idx keep=player school region seed ppg seed);
where school='Texas';
run;

/**/
title '6d. IDXWHERE on Player or School';
proc print data=ncaam06_update_b (idxwhere=yes keep=player school region seed ppg seed);
where player in ('Steve Burtt', 'Jared Dudley', 'Stanley Burrell') or school='Texas';
run;

/**/
title '6e. IDXWHERE on Player and School';
proc print data=ncaam06_update_b (idxwhere=yes keep=player school region seed ppg seed);
where substr(player, 1, 1) = 'S' and school in ('Duke', 'Oral Roberts', 'Iona', 'Boston College', 'Gonzaga');
run;


Obs,Seed,School,Region,Player,PPG
8,4,Boston Coll,MIN,Jared Dudley,16.5
193,14,Xavier,OAK,Stanley Burrell,14.1
57,13,Iona,ATL,Steve Burtt,25.2
Obs,Seed,School,Region,Player,PPG
8,4,Boston Coll,MIN,Jared Dudley,16.5
57,13,Iona,ATL,Steve Burtt,25.2
152,2,Texas,ATL,P.J. Tucker,16.2
153,2,Texas,ATL,LaMarcus Aldridge,15.0
154,2,Texas,ATL,Daniel Gibson,13.7
155,2,Texas,ATL,Kenton Paulino,10.0


In [ ]:
title;
footnote;
ods pdf close;